O objetivo do código é realizar um merge entre um arquivo shapefile (limites municipais) com um arquivo csv (lista de municípos), onde, a única informação em comum entre os dois arquivos são os nomes dos municípios.

#### Importando as bibliotecas necessárias

In [1]:
# Realizamos a importação das bibliotecas necessárias. Importamos a pandas apenas para uma visualização e, se necessário, manipulação do arquivo csv.

import geopandas as gpd
import pandas as pd

#### Carregando e manipulando o arquivo csv.

In [2]:
# Carregando e fazendo uma visualização prévia do conteúdo do arquivo csv

csv = pd.read_csv('/home/rivas/Estudos/PythonGIS/data/relacao_municipios.csv', header=0) # Acessando o arquivo

csv.head() # Visualizando os cinco primeiros registros

,id,mun
0,1,Anagé
1,2,Andaraí
2,3,Andorinha
3,4,Aracatu
4,5,Araci


In [3]:
csv['emer_seca'] = 'Sim' # Aqui criamos o campo 'emer_seca' e atribuímos o valor 'Sim' aos municípios em siuação de emergência.

csv.head() # Mais uma vez visualizando os cinco primeiros registros

,id,mun,emer_seca
0,1,Anagé,Sim
1,2,Andaraí,Sim
2,3,Andorinha,Sim
3,4,Aracatu,Sim
4,5,Araci,Sim


In [4]:
csv.shape # Com a instrução shape podemos ter a informação da "forma" da tabela csv que possui 95 linhas e 3 colunas.

(95, 3)

#### Carregando e manipulando o arquivo shapefile.

In [5]:
shp = gpd.read_file('/home/rivas/Estudos/PythonGIS/data/mun_limites.shp')

shp.head()

,OBJECTID,CD_MUN,NM_MUN,Territorio,PopRes,Est_Agrico,Est_AgricF,BiomaPredo,Shape_Leng,geometry
0,2,2900207,Abaré,Itaparica,16991,1885,1700,Caatinga,1.708172,"POLYGON ((-39.35855 -8.54733, -39.35830 -8.547..."
1,5,2900405,Água Fria,Portal do Sertão,15659,2165,1705,Caatinga e Mata Atlântica,1.129475,"POLYGON ((-38.64762 -11.69026, -38.64682 -11.6..."
2,14,2901205,Anagé,Sudoeste Baiano,25489,2475,2065,Caatinga e Mata Atlântica,2.296842,"POLYGON ((-40.71652 -14.38046, -40.71652 -14.3..."
3,26,2902104,Araci,Sisal,51536,4779,3635,Caatinga,2.061252,"POLYGON ((-39.26250 -10.99298, -39.26168 -10.9..."
4,33,2902658,Banzaê,Semiárido Nordeste II,11801,1754,1590,Caatinga,0.923486,"POLYGON ((-38.66991 -10.48238, -38.66963 -10.4..."


In [6]:
shp.shape

(417, 10)

#### Realizando o merge

In [7]:
# Para a realização do merge serão utilizados os campos NM_MUN da camada mun_limites e o campo mun da lisat csv.

uniao = shp.merge(csv, left_on= 'NM_MUN', right_on='mun')


uniao.head()

,OBJECTID,CD_MUN,NM_MUN,Territorio,PopRes,Est_Agrico,Est_AgricF,BiomaPredo,Shape_Leng,geometry,id,mun,emer_seca
0,14,2901205,Anagé,Sudoeste Baiano,25489,2475,2065,Caatinga e Mata Atlântica,2.296842,"POLYGON ((-40.71652 -14.38046, -40.71652 -14.3...",1,Anagé,Sim
1,26,2902104,Araci,Sisal,51536,4779,3635,Caatinga,2.061252,"POLYGON ((-39.26250 -10.99298, -39.26168 -10.9...",5,Araci,Sim
2,47,2903805,Boa Vista do Tupim,Piemonte do Paraguaçu,17917,2536,1901,Caatinga,2.612894,"POLYGON ((-40.56749 -12.45239, -40.56697 -12.4...",10,Boa Vista do Tupim,Sim
3,49,2903953,Bom Jesus da Serra,Sudoeste Baiano,10111,1570,1433,Caatinga e Mata Atlântica,1.087668,"POLYGON ((-40.67763 -14.32803, -40.67726 -14.3...",12,Bom Jesus da Serra,Sim
4,56,2904506,Brotas de Macaúbas,Velho Chico,10697,1640,1579,Caatinga,3.138704,"POLYGON ((-42.75099 -11.72904, -42.75076 -11.7...",13,Brotas de Macaúbas,Sim


In [8]:
uniao.shape # Após o merge verificamos que tivemos 10 municipios a menos em relação aos 95 listados no arquivo csv.

(85, 13)

Como é comum, ao se realizar operações de merge/join utilizando coampos de texto a chances de ocorrer erro é maior. Podem ocorrer erros por diversos fatores como escrita de formas diferentes, palavras com acentuação gráfica e outras sem... e por aí vai.

Agora precisaremos identificar os campos que não forma unidos para verificarmos quais os erros e realizar alguma correção.

Para isso iremos refazer o merge mas adicionando alguns parâmetros que poderão ajudar na identificação nos campos com problema.

In [9]:
# Realizando o merge

uniao = shp.merge(csv, left_on= 'NM_MUN', right_on='mun', how='outer', indicator=True) #Com os parâmetros 'outer' e 'indicator' podemos identificar os campos que não tiveram correspondência


uniao.head()

,OBJECTID,CD_MUN,NM_MUN,Territorio,PopRes,Est_Agrico,Est_AgricF,BiomaPredo,Shape_Leng,geometry,id,mun,emer_seca,_merge
0,2.0,2900207,Abaré,Itaparica,16991.0,1885.0,1700.0,Caatinga,1.708172,"POLYGON ((-39.35855 -8.54733, -39.35830 -8.547...",NaN,NaN,NaN,left_only
1,5.0,2900405,Água Fria,Portal do Sertão,15659.0,2165.0,1705.0,Caatinga e Mata Atlântica,1.129475,"POLYGON ((-38.64762 -11.69026, -38.64682 -11.6...",NaN,NaN,NaN,left_only
2,14.0,2901205,Anagé,Sudoeste Baiano,25489.0,2475.0,2065.0,Caatinga e Mata Atlântica,2.296842,"POLYGON ((-40.71652 -14.38046, -40.71652 -14.3...",1.0,Anagé,Sim,both
3,26.0,2902104,Araci,Sisal,51536.0,4779.0,3635.0,Caatinga,2.061252,"POLYGON ((-39.26250 -10.99298, -39.26168 -10.9...",5.0,Araci,Sim,both
4,33.0,2902658,Banzaê,Semiárido Nordeste II,11801.0,1754.0,1590.0,Caatinga,0.923486,"POLYGON ((-38.66991 -10.48238, -38.66963 -10.4...",NaN,NaN,NaN,left_only


In [10]:
uniao[uniao["_merge"] != "both"] # Verifcação se tivemos ou não campos sem correspondncia. Qualquer valor diferente de "both" significa que a correspondência foi unilateral.

,OBJECTID,CD_MUN,NM_MUN,Territorio,PopRes,Est_Agrico,Est_AgricF,BiomaPredo,Shape_Leng,geometry,id,mun,emer_seca,_merge
0,2.0,2900207,Abaré,Itaparica,16991.0,1885.0,1700.0,Caatinga,1.708172,"POLYGON ((-39.35855 -8.54733, -39.35830 -8.547...",NaN,NaN,NaN,left_only
1,5.0,2900405,Água Fria,Portal do Sertão,15659.0,2165.0,1705.0,Caatinga e Mata Atlântica,1.129475,"POLYGON ((-38.64762 -11.69026, -38.64682 -11.6...",NaN,NaN,NaN,left_only
4,33.0,2902658,Banzaê,Semiárido Nordeste II,11801.0,1754.0,1590.0,Caatinga,0.923486,"POLYGON ((-38.66991 -10.48238, -38.66963 -10.4...",NaN,NaN,NaN,left_only
5,38.0,2903102,Barra do Rocha,Médio Rio de Contas,6263.0,461.0,296.0,Mata Atlântica,0.936836,"POLYGON ((-39.54388 -13.94903, -39.54394 -13.9...",NaN,NaN,NaN,left_only
6,42.0,2903300,Barro Preto,Litoral Sul,6447.0,393.0,257.0,Mata Atlântica,0.735178,"POLYGON ((-39.39394 -14.70161, -39.39384 -14.7...",NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,66.0,Presidente Jânio quadros,Sim,right_only
423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,72.0,Santa Brigida,Sim,right_only
424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,80.0,Sitio do Mato,Sim,right_only
425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,87.0,Vitoria da Conquista,Sim,right_only


O resultado acima serviu para evidenciar que tivemos valores que só existiam ou na esquerda ou na direita.
Neste caso devemos corrigir todos os valores marcados com 'right_only' e refazer o merge de forma a termos
todos os 95 municípios da lista (variável csv) com correspondência na camada dos limites municipais (variável shp).

In [11]:
# Utilizando o loc para filtrar todos os municípios com valor 'right_only'.

uniao.loc[(uniao['_merge'] == 'right_only')]

,OBJECTID,CD_MUN,NM_MUN,Territorio,PopRes,Est_Agrico,Est_AgricF,BiomaPredo,Shape_Leng,geometry,id,mun,emer_seca,_merge
417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,18.0,Candido Sales,Sim,right_only
418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,38.0,Itiuba,Sim,right_only
419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,44.0,Livramento de N° Senhora,Sim,right_only
420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,51.0,Marcionilio de Souza,Sim,right_only
421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,55.0,Muquém de São Francisco,Sim,right_only
422,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,66.0,Presidente Jânio quadros,Sim,right_only
423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,72.0,Santa Brigida,Sim,right_only
424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,80.0,Sitio do Mato,Sim,right_only
425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,87.0,Vitoria da Conquista,Sim,right_only
426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,92.0,Palmas de MOnte Alto,Sim,right_only


#### Corrigindo nomes dos municípios

Agora que identificamos quais linhas estão com erros nos nomes, podemos utilizar o replace() para realizar a correção.

In [12]:
# Testando a correção com a função replace().

csv_corrercao = csv
csv_corrercao['mun'].replace('Candido Sales', 'Cândido Sales', inplace=True)
csv_corrercao.loc[(csv_corrercao['mun'] == 'Cândido Sales')]


,id,mun,emer_seca
17,18,Cândido Sales,Sim


In [13]:
# Corrigindo todos os demais municípios com a função replace().

csv_corrercao['mun'].replace('Itiuba', 'Itiúba', inplace=True)
csv_corrercao['mun'].replace('Livramento de N° Senhora', 'Livramento de Nossa Senhora', inplace=True)
csv_corrercao['mun'].replace('Marcionilio de Souza', 'Marcionílio Souza', inplace=True)
csv_corrercao['mun'].replace('Muquém de São Francisco', 'Muquém do São Francisco', inplace=True)
csv_corrercao['mun'].replace('Presidente Jânio quadros', 'Presidente Jânio Quadros', inplace=True)
csv_corrercao['mun'].replace('Santa Brigida', 'Santa Brígida', inplace=True)
csv_corrercao['mun'].replace('Sitio do Mato', 'Sítio do Mato', inplace=True)
csv_corrercao['mun'].replace('Vitoria da Conquista', 'Vitória da Conquista', inplace=True)
csv_corrercao['mun'].replace('Palmas de MOnte Alto', 'Palmas de Monte Alto', inplace=True)

Com os nomes corrigidos na lista csv podemos refazer o merge com a camada shapefile.

In [14]:
# Refazendo a união

uniao = shp.merge(csv, left_on= 'NM_MUN', right_on='mun', how='outer', indicator=True) # Realizando o merge mas identificando os campos que não tiveram correspondência

uniao.head()

,OBJECTID,CD_MUN,NM_MUN,Territorio,PopRes,Est_Agrico,Est_AgricF,BiomaPredo,Shape_Leng,geometry,id,mun,emer_seca,_merge
0,2,2900207,Abaré,Itaparica,16991,1885,1700,Caatinga,1.708172,"POLYGON ((-39.35855 -8.54733, -39.35830 -8.547...",NaN,NaN,NaN,left_only
1,5,2900405,Água Fria,Portal do Sertão,15659,2165,1705,Caatinga e Mata Atlântica,1.129475,"POLYGON ((-38.64762 -11.69026, -38.64682 -11.6...",NaN,NaN,NaN,left_only
2,14,2901205,Anagé,Sudoeste Baiano,25489,2475,2065,Caatinga e Mata Atlântica,2.296842,"POLYGON ((-40.71652 -14.38046, -40.71652 -14.3...",1.0,Anagé,Sim,both
3,26,2902104,Araci,Sisal,51536,4779,3635,Caatinga,2.061252,"POLYGON ((-39.26250 -10.99298, -39.26168 -10.9...",5.0,Araci,Sim,both
4,33,2902658,Banzaê,Semiárido Nordeste II,11801,1754,1590,Caatinga,0.923486,"POLYGON ((-38.66991 -10.48238, -38.66963 -10.4...",NaN,NaN,NaN,left_only


In [15]:
# Checando se algum valor para 'mun' ficou sem correspondência

uniao.loc[(uniao['_merge'] == 'both')]

,OBJECTID,CD_MUN,NM_MUN,Territorio,PopRes,Est_Agrico,Est_AgricF,BiomaPredo,Shape_Leng,geometry,id,mun,emer_seca,_merge
2,14,2901205,Anagé,Sudoeste Baiano,25489,2475,2065,Caatinga e Mata Atlântica,2.296842,"POLYGON ((-40.71652 -14.38046, -40.71652 -14.3...",1.0,Anagé,Sim,both
3,26,2902104,Araci,Sisal,51536,4779,3635,Caatinga,2.061252,"POLYGON ((-39.26250 -10.99298, -39.26168 -10.9...",5.0,Araci,Sim,both
7,47,2903805,Boa Vista do Tupim,Piemonte do Paraguaçu,17917,2536,1901,Caatinga,2.612894,"POLYGON ((-40.56749 -12.45239, -40.56697 -12.4...",10.0,Boa Vista do Tupim,Sim,both
8,49,2903953,Bom Jesus da Serra,Sudoeste Baiano,10111,1570,1433,Caatinga e Mata Atlântica,1.087668,"POLYGON ((-40.67763 -14.32803, -40.67726 -14.3...",12.0,Bom Jesus da Serra,Sim,both
9,56,2904506,Brotas de Macaúbas,Velho Chico,10697,1640,1579,Caatinga,3.138704,"POLYGON ((-42.75099 -11.72904, -42.75076 -11.7...",13.0,Brotas de Macaúbas,Sim,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,384,2931053,Tanque Novo,Sertão Produtivo,16108,2612,2267,Caatinga,1.326464,"POLYGON ((-42.62353 -13.40672, -42.58339 -13.4...",82.0,Tanque Novo,Sim,both
402,395,2932002,Uauá,Sertão do São Francisco,24273,3215,2917,Caatinga,2.332941,"POLYGON ((-39.32506 -9.59735, -39.32384 -9.597...",84.0,Uauá,Sim,both
406,402,2932606,Urandi,Sertão Produtivo,16447,2076,1771,Cerrado e Caatinga,1.494008,"POLYGON ((-42.56093 -14.52860, -42.56219 -14.5...",85.0,Urandi,Sim,both
409,406,2933000,Valente,Sisal,24544,1511,1191,Caatinga,0.891934,"POLYGON ((-39.33191 -11.34150, -39.32531 -11.3...",86.0,Valente,Sim,both


#### Exportando para um arquivo shapefile.

In [16]:
# Removendo a coluna '_merge' com a função drop().

shp_seca = uniao

shp_seca = shp_seca.drop(columns=['_merge'])

shp_seca

,OBJECTID,CD_MUN,NM_MUN,Territorio,PopRes,Est_Agrico,Est_AgricF,BiomaPredo,Shape_Leng,geometry,id,mun,emer_seca
0,2,2900207,Abaré,Itaparica,16991,1885,1700,Caatinga,1.708172,"POLYGON ((-39.35855 -8.54733, -39.35830 -8.547...",NaN,NaN,NaN
1,5,2900405,Água Fria,Portal do Sertão,15659,2165,1705,Caatinga e Mata Atlântica,1.129475,"POLYGON ((-38.64762 -11.69026, -38.64682 -11.6...",NaN,NaN,NaN
2,14,2901205,Anagé,Sudoeste Baiano,25489,2475,2065,Caatinga e Mata Atlântica,2.296842,"POLYGON ((-40.71652 -14.38046, -40.71652 -14.3...",1.0,Anagé,Sim
3,26,2902104,Araci,Sisal,51536,4779,3635,Caatinga,2.061252,"POLYGON ((-39.26250 -10.99298, -39.26168 -10.9...",5.0,Araci,Sim
4,33,2902658,Banzaê,Semiárido Nordeste II,11801,1754,1590,Caatinga,0.923486,"POLYGON ((-38.66991 -10.48238, -38.66963 -10.4...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,409,2933158,Várzea Nova,Piemonte da Diamantina,13049,1542,1395,Caatinga,1.521162,"POLYGON ((-41.23382 -10.95670, -41.13406 -10.9...",NaN,NaN,NaN
413,410,2933174,Varzedo,Recôncavo,9108,1354,1163,Mata Atlântica,0.755003,"POLYGON ((-39.32318 -12.99110, -39.32298 -12.9...",NaN,NaN,NaN
414,412,2933257,Vereda,Extremo Sul,6786,284,194,Mata Atlântica,3.180570,"POLYGON ((-40.40267 -16.89737, -40.40300 -16.8...",NaN,NaN,NaN
415,415,2933455,Wanderley,Bacia do Rio Grande,12450,1660,1053,Cerrado,3.272708,"POLYGON ((-43.82126 -11.37674, -43.81968 -11.3...",NaN,NaN,NaN


In [17]:
# Exportando oresutado final para arquivo shapefile.

shp_seca.to_file('/home/rivas/Estudos/PythonGIS/data/mun_seca2023.shp')